In [1]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')

# For importing .xml files
import xml.etree.ElementTree as ET

# For handeling dataframes
import pandas as pd
import numpy as np
import re                                  # library for regular expression operations
import string                              # for string operations

# For text preprocessing
import nltk                                # Natural Language Toolkit
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer as ps  # module for stemming

# For building neural netwrok models
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers, losses
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import transformers

# For model evaluation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score
import time


In [2]:
# Load the data
labels_all = pd.read_csv("data/labels_all.csv")
labels_all.drop(['Unnamed: 0'], axis=1, inplace=True)

text_all = pd.read_csv('data/text_all.csv')
text_all.drop(['Unnamed: 0'], axis=1, inplace=True)

#  Create binary labels for id
disease_types = ['Hypertriglyceridemia','Venous Insufficiency','Asthma','Gout','OSA','PVD','Gallstones','OA','GERD',
                'Depression','Obesity','CHF','Hypercholesterolemia','CAD','Diabetes','Hypertension']

In [3]:
def make_binary(labels_df):
    for i in range(16):
        # In the disease column, code the disease of interest as 1 and other disease types as 0 to indicate 
        # whether this row is about this particular disease
        labels_df["disease_"+str(i)] = [1 if x==disease_types[i] else 0 for x in labels_df["disease"]]
        # In the label column, code Y as 1 and everything else as 0 to indicate whether a disease is present
        # (it may or may not be about this particular disease)
        labels_df["label_"+str(i)] = [1 if x=="Y" else 0 for x in labels_df["label"]] 
        # Create a "This_Disease" column that indicates whether a patient has this particular disease or not
        # A patient is coded as having this disease only when (disease is 1) AND (label is 1)
        labels_df["This_Disease_"+str(i)] = labels_df["disease_"+str(i)] * labels_df["label_"+str(i)] 
        labels_df.drop(["disease_"+str(i), "label_"+str(i)], axis=1, inplace=True)

    # Now that all useful info in "disease" and "Label" are combined in column "This_Disease", no longer need disease and label
    labels_df.drop(['label', 'disease'], axis=1, inplace=True)
    # Duplicates after removing disease and label are due to 0's in the "disease" column if the patient had other disease info
    labels_df.drop_duplicates(inplace = True)
    # Aggregate the binary values into one row for each id
    labels_df = labels_df.groupby('id').aggregate('sum').reset_index()
    
    return(labels_df)

In [4]:
binary_labels = make_binary(labels_all)

# Calculate the disease prevalence for each disease type
prevalence = binary_labels.aggregate('sum')/1237


# Combine the x_test and y_test sets
df = text_all.merge(binary_labels, on="id", how="left")

In [5]:
# Download the stopwords from NLTK
nltk.download('stopwords')

# Import the standard English stop words list from NLTK
stopwords_english = stopwords.words('english') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
additional_sw = ['admission', 'date', 'discharge', 'summary', 'diagnoses', 'diagnosis', 'history',
                    'patient', 'report', 'status', 'signed', 'attending', 'service', 'dictated', 'dictation']
all_stopwords = additional_sw + stopwords_english

In [7]:
# # Convert all words in text column to lower case in both the train and test sets
# df['text'] = [str(x).lower() for x in df['text']]

# # Remove noiseb
# df['text']  = [re.sub("(\W|\d+|\n)", " ", elem).strip() for elem in df['text']] # remove spaces, digits and line breaks

# # Remove punctuations
# def remove_punctuations(text):
#     return ' '.join(['' if (elem in string.punctuation) else elem for elem in text.split()])

# df['text'] = [remove_punctuations(elem) for elem in df['text']]


# # Remove standard English stop words
# def remove_stopwords(text):
#     return ' '.join(['' if (elem in all_stopwords) else elem for elem in text.split()])

# df['text'] = [remove_stopwords(elem) for elem in df['text']]

# # Remove words with a length of one or two
# def remove_short_words(text):
#     return ' '.join(['' if (len(elem) <= 2) else elem for elem in text.split()])

# df['text'] = [remove_short_words(elem) for elem in df['text']]


In [8]:
def preprocessing(text):
    text = "".join([x for x in text if x not in string.punctuation]) # Remmove punctuations 
    text = "".join([x.lower() for x in text]) # Convert to lower case
    text = ' '.join(['' if (x in all_stopwords) else x for x in text.split()]) # Remove stopwords
    text = re.sub("(\W|\d+|\n)", " ", text).strip() # remove spaces, digits and line breaks
    text = ' '.join(['' if (len(x) <= 2) else x for x in text.split()]) # Remove short words
    return(text)

In [9]:
df['text'] = [preprocessing(x) for x in df['text']]

In [10]:
# Define a function to evaluate the model performance in terms of F1 score
def evaluate(model, X, y):
    pred = model.predict(X)
    pred = [1 if p>=0.5 else 0 for p in pred]
    acc = np.sum(y == pred)/len(y)
    
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()

    precision = tp/(tp + fp)
    recall = tp/(tp + fn)
    specificity = tn/(tn + fp)
    f1 = (2*precision*recall)/(precision + recall)
    
    y_pred = model.predict(X)
    auc_roc = round(roc_auc_score(y, y_pred),4)
    pre, rec, thresholds = precision_recall_curve(y, y_pred)
    auc_pr = round(auc(rec, pre),4)
    return(auc_roc, auc_pr, acc, precision, recall, specificity, f1)

In [11]:
# Create the CNN model
def cnnmodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(Conv1D(8, kernel_size=8, activation="relu"))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())  
    model.add(Dense(units=1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [12]:
# Create the RNN model
def rnnmodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.SimpleRNN(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [13]:
# Create the GRU model
def grumodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential()   
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.GRU(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [14]:
# Create the LSTM model
def lstmmodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential() 
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.LSTM(units=8)) 
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [15]:
# Create the LSTM model
def bi_lstm_model(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix):    
    model = Sequential() 
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length,
                              weights=[embedding_matrix],trainable=False))
    model.add(layers.Bidirectional(layers.LSTM(8)))
    model.add(Dropout(dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    opt = keras.optimizers.Adam()
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['AUC'])
    return(model)

In [17]:
# Load the GloVe word embeddings, and append them to a dictionary
glove_index = {}
f = open('word_embeddings/glove_6B/glove.6B.200d.txt', encoding="utf8") # Glove word embeddings

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove_index[word] = coefs
f.close()

In [20]:
# Create an embedding matrix for each word in the training set
def emb_matrix(word_index):
    count = 0
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = glove_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
        else:
            count += 1
    return embedding_matrix

In [22]:
# Set the number of epochs and the batch size
epochs = 10 
batch_size = 32
max_length = 557
embedding_dim = 200
dropout_rate=0.3

In [23]:
# Run the models
X = df['text']
models = [cnnmodel,rnnmodel,grumodel,lstmmodel,bi_lstm_model]
model_names = ['cnnmodel','rnnmodel','grumodel','lstmmodel','bi_lstm_model']

time_1 = time.time()

for i in range(16): # Go through each disease type

    event_categorical = df['This_Disease_'+str(i)]
    dis_type = disease_types[i]
    
    time_start = time.time()   

    # Split the data 10 times into train and test sets with stratification
    sss = StratifiedShuffleSplit(n_splits=10, test_size=0.25,random_state=0)
    
    j = 0
    for train_index, test_index in sss.split(X, event_categorical):   
               
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = event_categorical[train_index], event_categorical[test_index]
        
        tokenizer = Tokenizer(num_words=5000) # Tokenize the text
        tokenizer.fit_on_texts(x_train)
        x_train = tokenizer.texts_to_sequences(x_train)
        x_test = tokenizer.texts_to_sequences(x_test)
        word_index = tokenizer.word_index
        vocab_size = len(word_index) + 1 # plus the reserved index 0
        

        # Pad the sequences with 0's
        x_train = pad_sequences(x_train, padding='post', maxlen=max_length) 
        x_test = pad_sequences(x_test, padding='post', maxlen=max_length)
        
        embedding_matrix = emb_matrix(word_index)

        # Fit the models
        j += 1
        iteration = "iter" + str(j)
        
        for m in range(len(models)):    
            time_s = time.time()  
            mymodel = models[m]  
            this_model = mymodel(vocab_size, embedding_dim,max_length,dropout_rate,embedding_matrix)
            this_model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)
            time_e = time.time() - time_s

            # Collect and log evaluation metrics
            auc_roc, auc_pr, acc, precision, recall, specificity, f1 = evaluate(this_model, x_test, y_test)
            # Save the model performance metrics
            with open('other/CNN_Paper_Glove.csv','a') as fd:
                fd.write(f'{dis_type},{iteration},{model_names[m]},{auc_roc},{auc_pr},{acc},{precision},{recall},{specificity},{f1},{time_e},{vocab_size},{len(x_train)}\n')
    
    # Average metrics
    running_time = time.time() - time_start

    with open('other/CNN_Paper_Glove.csv','a') as fd:
        fd.write(f'{dis_type}, 10 iteration running time, {running_time}\n')

Epoch 1/10
29/29 [==============================] - 5s 90ms/step - loss: 0.2562 - auc: 0.4653
Epoch 2/10
29/29 [==============================] - 3s 90ms/step - loss: 0.1812 - auc: 0.7810: 1s - loss: 0.1866 - auc: 0. - ETA: 0s - loss: 0.1732 - 
Epoch 3/10
29/29 [==============================] - 3s 96ms/step - loss: 0.1266 - auc: 0.9631
Epoch 4/10
29/29 [==============================] - 3s 92ms/step - loss: 0.0790 - auc: 0.9969: 0s - loss: 0.0622 - a
Epoch 5/10
29/29 [==============================] - 3s 96ms/step - loss: 0.0629 - auc: 0.9979
Epoch 6/10
29/29 [==============================] - 3s 89ms/step - loss: 0.0340 - auc: 0.9999: 1s - loss: 0.037
Epoch 7/10
29/29 [==============================] - 3s 90ms/step - loss: 0.0214 - auc: 1.0000: 0s - loss: 0.0210 - auc: 1
Epoch 8/10
29/29 [==============================] - 3s 93ms/step - loss: 0.0169 - auc: 1.0000
Epoch 9/10
29/29 [==============================] - 3s 98ms/step - loss: 0.0123 - auc: 1.0000: 0s - loss: 0.0122 - auc: 
E

29/29 [==============================] - 5s 168ms/step - loss: 0.2263 - auc: 0.6024
Epoch 10/10
29/29 [==============================] - 5s 161ms/step - loss: 0.2236 - auc: 0.5819
Epoch 1/10
29/29 [==============================] - 15s 311ms/step - loss: 0.5845 - auc: 0.3902
Epoch 2/10
29/29 [==============================] - 9s 301ms/step - loss: 0.4894 - auc: 0.3970
Epoch 3/10
29/29 [==============================] - 9s 309ms/step - loss: 0.4201 - auc: 0.4155
Epoch 4/10
29/29 [==============================] - 9s 308ms/step - loss: 0.3254 - auc: 0.4039
Epoch 5/10
29/29 [==============================] - 9s 303ms/step - loss: 0.2407 - auc: 0.5170
Epoch 6/10
29/29 [==============================] - 9s 315ms/step - loss: 0.2185 - auc: 0.5604
Epoch 7/10
29/29 [==============================] - 9s 300ms/step - loss: 0.2146 - auc: 0.5592
Epoch 8/10
29/29 [==============================] - 9s 311ms/step - loss: 0.2029 - auc: 0.6169
Epoch 9/10
29/29 [==============================] - 9s 302m

29/29 [==============================] - 8s 260ms/step - loss: 0.2372 - auc: 0.4271
Epoch 10/10
29/29 [==============================] - 8s 277ms/step - loss: 0.2259 - auc: 0.5165
Epoch 1/10
29/29 [==============================] - 20s 403ms/step - loss: 0.5604 - auc: 0.4436
Epoch 2/10
29/29 [==============================] - 12s 430ms/step - loss: 0.3218 - auc: 0.5058
Epoch 3/10
29/29 [==============================] - 12s 415ms/step - loss: 0.2482 - auc: 0.5103
Epoch 4/10
29/29 [==============================] - 13s 433ms/step - loss: 0.2257 - auc: 0.5165
Epoch 5/10
29/29 [==============================] - 12s 410ms/step - loss: 0.2206 - auc: 0.4983
Epoch 6/10
29/29 [==============================] - 12s 401ms/step - loss: 0.2108 - auc: 0.5874
Epoch 7/10
29/29 [==============================] - 11s 376ms/step - loss: 0.2059 - auc: 0.6427
Epoch 8/10
29/29 [==============================] - 11s 379ms/step - loss: 0.2059 - auc: 0.6354
Epoch 9/10
29/29 [==============================] - 

29/29 [==============================] - 4s 128ms/step - loss: 0.0215 - auc: 1.0000
Epoch 1/10
29/29 [==============================] - 6s 145ms/step - loss: 0.6349 - auc: 0.5144
Epoch 2/10
29/29 [==============================] - 4s 145ms/step - loss: 0.4363 - auc: 0.4354
Epoch 3/10
29/29 [==============================] - 5s 161ms/step - loss: 0.3490 - auc: 0.4822
Epoch 4/10
29/29 [==============================] - 5s 163ms/step - loss: 0.3159 - auc: 0.4797
Epoch 5/10
29/29 [==============================] - 4s 154ms/step - loss: 0.3073 - auc: 0.3987
Epoch 6/10
29/29 [==============================] - 4s 147ms/step - loss: 0.2869 - auc: 0.4511
Epoch 7/10
29/29 [==============================] - 5s 167ms/step - loss: 0.2778 - auc: 0.4980
Epoch 8/10
29/29 [==============================] - 4s 147ms/step - loss: 0.2711 - auc: 0.5052
Epoch 9/10
29/29 [==============================] - 4s 141ms/step - loss: 0.2608 - auc: 0.5601
Epoch 10/10
29/29 [==============================] - 5s 162ms

29/29 [==============================] - 8s 290ms/step - loss: 0.2534 - auc: 0.5563
Epoch 1/10
29/29 [==============================] - 11s 253ms/step - loss: 0.5927 - auc: 0.4698
Epoch 2/10
29/29 [==============================] - 8s 277ms/step - loss: 0.4641 - auc: 0.4561
Epoch 3/10
29/29 [==============================] - 8s 271ms/step - loss: 0.3096 - auc: 0.5011
Epoch 4/10
29/29 [==============================] - 9s 311ms/step - loss: 0.2845 - auc: 0.4461
Epoch 5/10
29/29 [==============================] - 8s 278ms/step - loss: 0.2627 - auc: 0.5444
Epoch 6/10
29/29 [==============================] - 8s 291ms/step - loss: 0.2663 - auc: 0.5085
Epoch 7/10
29/29 [==============================] - 8s 262ms/step - loss: 0.2509 - auc: 0.5821
Epoch 8/10
29/29 [==============================] - 8s 284ms/step - loss: 0.2553 - auc: 0.5387
Epoch 9/10
29/29 [==============================] - 7s 257ms/step - loss: 0.2444 - auc: 0.6196
Epoch 10/10
29/29 [==============================] - 8s 281m

...

Epoch 9/10
29/29 [==============================] - 3s 110ms/step - loss: 0.6126 - auc: 0.6054
Epoch 10/10
29/29 [==============================] - 3s 117ms/step - loss: 0.6143 - auc: 0.6225
Epoch 1/10
29/29 [==============================] - 9s 209ms/step - loss: 0.6648 - auc: 0.5627
Epoch 2/10
29/29 [==============================] - 7s 225ms/step - loss: 0.6434 - auc: 0.5705
Epoch 3/10
29/29 [==============================] - 6s 210ms/step - loss: 0.6363 - auc: 0.5749
Epoch 4/10
29/29 [==============================] - 6s 209ms/step - loss: 0.6277 - auc: 0.5885
Epoch 5/10
29/29 [==============================] - 6s 224ms/step - loss: 0.6265 - auc: 0.5788
Epoch 6/10
29/29 [==============================] - 6s 209ms/step - loss: 0.6243 - auc: 0.6013
Epoch 7/10
29/29 [==============================] - 6s 222ms/step - loss: 0.6190 - auc: 0.6221
Epoch 8/10
29/29 [==============================] - 6s 224ms/step - loss: 0.6187 - auc: 0.6108
Epoch 9/10
29/29 [==============================]

29/29 [==============================] - 6s 194ms/step - loss: 0.5947 - auc: 0.6654 1s - loss: 0.5907 - auc
Epoch 10/10
29/29 [==============================] - 6s 206ms/step - loss: 0.5871 - auc: 0.6772
Epoch 1/10
29/29 [==============================] - 11s 265ms/step - loss: 0.6769 - auc: 0.4993
Epoch 2/10
29/29 [==============================] - 8s 279ms/step - loss: 0.6241 - auc: 0.6146
Epoch 3/10
29/29 [==============================] - 8s 278ms/step - loss: 0.6094 - auc: 0.6546
Epoch 4/10
29/29 [==============================] - 8s 279ms/step - loss: 0.6008 - auc: 0.6723
Epoch 5/10
29/29 [==============================] - 8s 276ms/step - loss: 0.5885 - auc: 0.6997
Epoch 6/10
29/29 [==============================] - 8s 276ms/step - loss: 0.5748 - auc: 0.7203
Epoch 7/10
29/29 [==============================] - 8s 283ms/step - loss: 0.5649 - auc: 0.7365
Epoch 8/10
29/29 [==============================] - 8s 271ms/step - loss: 0.5498 - auc: 0.7615
Epoch 9/10
29/29 [=================

Epoch 5/10
29/29 [==============================] - 6s 222ms/step - loss: 0.5644 - auc: 0.6096
Epoch 6/10
29/29 [==============================] - 6s 213ms/step - loss: 0.5615 - auc: 0.6156
Epoch 7/10
29/29 [==============================] - 7s 226ms/step - loss: 0.5624 - auc: 0.6034
Epoch 8/10
29/29 [==============================] - 6s 218ms/step - loss: 0.5569 - auc: 0.6190
Epoch 9/10
29/29 [==============================] - 7s 228ms/step - loss: 0.5586 - auc: 0.5984
Epoch 10/10
29/29 [==============================] - 6s 217ms/step - loss: 0.5391 - auc: 0.6758
Epoch 1/10
29/29 [==============================] - 8s 207ms/step - loss: 0.6548 - auc: 0.4843
Epoch 2/10
29/29 [==============================] - 6s 204ms/step - loss: 0.6227 - auc: 0.5365
Epoch 3/10
29/29 [==============================] - 6s 191ms/step - loss: 0.6016 - auc: 0.5282
Epoch 4/10
29/29 [==============================] - 6s 205ms/step - loss: 0.5792 - auc: 0.5835
Epoch 5/10
29/29 [==============================]

29/29 [==============================] - 8s 287ms/step - loss: 0.5446 - auc: 0.6905
Epoch 7/10
29/29 [==============================] - 8s 280ms/step - loss: 0.5383 - auc: 0.6998
Epoch 8/10
29/29 [==============================] - 8s 288ms/step - loss: 0.5182 - auc: 0.7409
Epoch 9/10
29/29 [==============================] - 8s 274ms/step - loss: 0.5060 - auc: 0.7544
Epoch 10/10
29/29 [==============================] - 8s 291ms/step - loss: 0.4932 - auc: 0.7750
Epoch 1/10
29/29 [==============================] - 4s 109ms/step - loss: 0.6161 - auc: 0.5258
Epoch 2/10
29/29 [==============================] - 3s 112ms/step - loss: 0.4945 - auc: 0.7920
Epoch 3/10
29/29 [==============================] - 3s 119ms/step - loss: 0.4074 - auc: 0.9038
Epoch 4/10
29/29 [==============================] - 3s 117ms/step - loss: 0.3270 - auc: 0.9583
Epoch 5/10
29/29 [==============================] - 3s 112ms/step - loss: 0.2700 - auc: 0.9752
Epoch 6/10
29/29 [==============================] - 3s 112ms

29/29 [==============================] - 3s 110ms/step - loss: 0.5780 - auc: 0.6001
Epoch 7/10
29/29 [==============================] - 3s 110ms/step - loss: 0.5754 - auc: 0.5895
Epoch 8/10
29/29 [==============================] - 3s 116ms/step - loss: 0.5702 - auc: 0.5875
Epoch 9/10
29/29 [==============================] - 3s 111ms/step - loss: 0.5689 - auc: 0.6013
Epoch 10/10
29/29 [==============================] - 3s 110ms/step - loss: 0.5674 - auc: 0.5920
Epoch 1/10
29/29 [==============================] - 9s 223ms/step - loss: 0.6515 - auc: 0.5588
Epoch 2/10
29/29 [==============================] - 6s 214ms/step - loss: 0.6131 - auc: 0.5447
Epoch 3/10
29/29 [==============================] - 6s 211ms/step - loss: 0.5914 - auc: 0.5533
Epoch 4/10
29/29 [==============================] - 7s 228ms/step - loss: 0.5892 - auc: 0.5580
Epoch 5/10
29/29 [==============================] - 6s 211ms/step - loss: 0.5735 - auc: 0.5942
Epoch 6/10
29/29 [==============================] - 6s 224ms

In [24]:
time_2 = time.time() - time_1  

In [25]:
time_2
with open('other/CNN_Paper_Glove.csv','a') as fd:
    fd.write(f'{dis_type}, Total running time, {time_2}\n')